# 1. Objetivo
O objetivo deste notebook é reproduzir a metodologia do cálculo de consumo aparente, apresentado no artigo [Indicadores de Consumo Aparente de Bens Industriais: metodologia e resultados](http://www.ipeadata.gov.br/doc/td_2101.pdf).

**Autor:** Amilton Lobo Mendes Júnior
amilton.mendes@gmail.com

# 2. Preparação do ambiente


## 2.1 Instalação de bibliotecas utilizadas

In [75]:
!pip install pyIpeaData

##2.2. Import das bibliotecas utilizadas neste notebook





In [76]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from datetime import datetime
import pyIpeaData as ip

## 2.3 URL das tabelas de recursos e usos de 2021, armazenada no github.

In [77]:
url_recursos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab1_2021.xls'
url_usos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab2_2021.xls'


## 2.4 Dicionários com informações sobre os dados por setor

In [78]:
#Criar um mapa que faça a correlação entre um código de cnae e códigos para baixar a série de importação, exportação e a url para download de dados do sidra

mapa_cnae = {
    "2100": {  # CNAE 2100 - Fabricação de produtos farmacêuticos
        "importacao": "FUNCEX12_MQFARM2N12",
        "exportacao": "FUNCEX12_XQFARM2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129330/d/v12606%205"
    },
    "29" : {
        "importacao": "FUNCEX12_MQVEIC2N12",
        "exportacao": "FUNCEX12_XQVEIC2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129338/d/v12606%205"
    }
    # adicione mais CNAEs aqui...
}



## 2.5 Funções e classes utilizadas no notebook

In [105]:
#Carrega dados oferta
def carrega_dados_oferta():
  oferta = pd.read_excel(url_recursos, sheet_name='oferta',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  #Renomear as duas primeiras colunas que não foram carregadas por estarem mescladas
  oferta= oferta.rename(columns={oferta.columns[0] : 'Código do produto', oferta.columns[1] : 'Descrição do produto'})
  #Remover a primeira linha (vazia)
  oferta = oferta.iloc[1:]
  return oferta
def carrega_dados_producao():
  producao = pd.read_excel(url_recursos, sheet_name='producao',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  producao = producao.rename(columns={producao.columns[0] : 'Código do produto', producao.columns[1] : 'Descrição do produto'})
  producao = producao.iloc[1:]
  return producao

def carrega_dados_importacao():
  importacao = pd.read_excel(url_recursos, sheet_name='importacao',skiprows=3,skipfooter=5,dtype={'Unnamed: 0' : str})
  importacao = importacao.rename(columns={importacao.columns[0] : 'Código do produto', importacao.columns[1] : 'Descrição do produto'})
  importacao = importacao.iloc[1:]
  return importacao

def carrega_dados_exportacao():
  exportacao = pd.read_excel(url_usos, sheet_name='demanda',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  exportacao = exportacao.rename(columns={exportacao.columns[0] : 'Código do produto', exportacao.columns[1] : 'Descrição do produto'})
  exportacao = exportacao.iloc[1:]
  return exportacao

class ConsumoAparente():
  def __init__(self,oferta,producao,importacao,exportacao):
    self.oferta = oferta
    self.producao = producao
    self.importacao = importacao
    self.exportacao = exportacao
  def carrega_dados_producao_mensal(self,url):
    r = requests.get(url)
    #Apresentar data de carregamento
    a = json.loads(r.text)
    res = json_normalize(a)

    producao = pd.DataFrame(res)
    nomes_colunas = producao.iloc[0].values
    producao.columns = nomes_colunas
    producao = producao.iloc[1:]
    producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
    producao['Ano'] = producao['Mês (Código)'].str[:4]
    producao['Mes'] = producao['Mês (Código)'].str[4:]

    producao['Ano'] = producao['Ano'].astype(int)
    producao['Mes'] = producao['Mes'].astype(int)

    producao = producao[['Ano','Mes','Valor']]
    producao = producao.rename(columns={'Valor' : 'producao'})

    producao['producao'] = producao['producao'].astype(float)

    return producao.copy()
  def carrega_dados_quantum_exp_imp(self,url,movimento):
    ##Carrega dados de quantum de exportação e importação
    dados_movimento = ip.get_serie(url)

    dados_movimento['VALDATA'] = pd.to_datetime(dados_movimento['VALDATA'],utc=True)

    dados_movimento['Ano'] = dados_movimento['VALDATA'].dt.year
    dados_movimento['Mes'] = dados_movimento['VALDATA'].dt.month

    dados_movimento = dados_movimento[['Ano','Mes','VALVALOR']]

    dados_movimento = dados_movimento.rename(columns={'VALVALOR' : movimento})
    dados_movimento[movimento] = dados_movimento[movimento].astype(float)

    return dados_movimento

  def calcula_consumo_aparente(self,cnae,verbose=0):
    preco_consumidor,preco_basico = self.oferta[self.oferta['Código do produto'].str.startswith(cnae)][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].sum().values
    fator_ajuste = preco_basico/preco_consumidor
    valor_producao = self.producao[self.producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].sum().values[0]
    valor_exportacao = self.exportacao[self.exportacao['Código do produto'].str.startswith(cnae)][['Exportação\nde bens e\nserviços (1)']].sum().values[0]
    valor_exportacao = valor_exportacao*fator_ajuste
    valor_importacao = self.importacao[self.importacao['Código do produto'].str.startswith(cnae)][['Importação de bens\ne serviços (1)']].sum().values[0]
    #Calcular pesos de acordo com fórmula constante do arquivo
    total = valor_importacao - valor_exportacao + valor_producao
    peso_importacao = valor_importacao/total
    peso_exportacao = valor_exportacao/total
    peso_producao = valor_producao/total

    #Prepara dados de quantum de exportação e importação para a junção com dados de produção e cálculo de consumo aparente
    if verbose == 1:
      print(f'Preço de consumidor: {preco_consumidor}')
      print(f'Preço básico: {preco_basico}')
      print(f'Fator de ajuste: {fator_ajuste}')
      print(f'Peso da importação: {peso_importacao}')
      print(f'Peso da exportação: {peso_exportacao}')
      print(f'Peso da produção: {peso_producao}')
      print('Carregando dados de produção da PIM-PF (Sidra)')
    producao = self.carrega_dados_producao_mensal(mapa_cnae[cnae]['sidra_url'])
    if verbose == 1:
      print('Carregando dados de exportação e importação da Quantum ( Funcex - Ipeadata)')
    dados_exportacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[cnae]['exportacao'],'exportacao')
    dados_importacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[cnae]['importacao'],'importacao')


    producao = producao.merge(dados_exportacao,how='inner',on=['Ano','Mes']).merge(dados_importacao,how='inner',on=['Ano','Mes'])

    producao['producao'] = producao['producao'].astype(float)
    producao['importacao'] = producao['importacao'].astype(float)
    producao['exportacao'] = producao['exportacao'].astype(float)

    producao['consumo_aparente'] = producao['producao']*peso_producao - producao['exportacao']*peso_exportacao + producao['importacao']*peso_importacao

    producao['consumo_aparente_12_meses_anterior'] = producao['consumo_aparente'].shift(12)

    producao['variacao_consumo_aparente_arredondado'] = ((producao['consumo_aparente']/producao['consumo_aparente_12_meses_anterior'])-1)*100

    return producao






In [106]:
consumoAparente = ConsumoAparente(carrega_dados_oferta(),carrega_dados_producao(),carrega_dados_importacao(),carrega_dados_exportacao())

In [107]:
consumoAparente.calcula_consumo_aparente('2100',1)

Preço de consumidor: 280596.0
Preço básico: 185107.0
Fator de ajuste: 0.6596922265463513
Peso da importação: 0.42250981749526956
Peso da exportação: 0.02206616206135398
Peso da produção: 0.5995563445660844
Carregando dados de produção da PIM-PF (Sidra)
Carregando dados de exportação e importação da Quantum ( Funcex - Ipeadata)


,Ano,Mes,producao,exportacao,importacao,consumo_aparente,consumo_aparente_12_meses_anterior,variacao_consumo_aparente_arredondado
0,2002,1,55.42792,19.73,32.93,46.710044,NaN,NaN
1,2002,2,76.43190,18.53,29.31,57.800107,NaN,NaN
2,2002,3,86.01877,21.86,40.93,68.384060,NaN,NaN
3,2002,4,91.12036,29.63,42.62,71.985338,NaN,NaN
4,2002,5,78.10670,23.99,33.14,60.301976,NaN,NaN
...,...,...,...,...,...,...,...,...
272,2024,9,102.27364,107.38,145.04,120.230169,115.089145,4.466993
273,2024,10,109.16742,104.73,167.95,134.101554,129.715955,3.380925
274,2024,11,101.50792,116.98,140.18,117.505844,112.793050,4.178266
275,2024,12,75.63582,101.94,141.96,103.078005,85.040120,21.211030


In [108]:
consumoAparente.calcula_consumo_aparente('29',0)

,Ano,Mes,producao,exportacao,importacao,consumo_aparente,consumo_aparente_12_meses_anterior,variacao_consumo_aparente_arredondado
0,2002,1,72.60072,33.81,24.00,65.405677,NaN,NaN
1,2002,2,74.06187,45.88,25.03,65.633879,NaN,NaN
2,2002,3,85.37216,55.18,30.07,75.696274,NaN,NaN
3,2002,4,91.68735,65.00,32.27,80.677540,NaN,NaN
4,2002,5,85.92352,61.88,33.43,76.246412,NaN,NaN
...,...,...,...,...,...,...,...,...
272,2024,9,115.60123,93.22,126.33,120.492563,101.674471,18.508177
273,2024,10,125.00663,97.25,156.81,135.387141,102.609210,31.944434
274,2024,11,108.98648,87.72,133.22,116.908653,101.722274,14.929256
275,2024,12,88.82298,85.94,118.75,96.110843,88.230995,8.930929


In [66]:
#calcular a variação do ano anterior, ou seja, 12 meses antes do mês de referência
producao['consumo_aparente_12_meses_arredondado'] = producao['consumo_aparente_arredondado'].shift(12)
producao['variacao_consumo_aparente_arredondado'] = ((producao['consumo_aparente_arredondado']/producao['consumo_aparente_12_meses_arredondado'])-1)*100
producao.tail(13)

,Ano,Mes,producao,importacao,exportacao,consumo_aparente,consumo_aparente_arredondado,consumo_aparente_12_meses_arredondado,variacao_consumo_aparente_arredondado
264,2024,1,72.82,148.36,80.55,104.3922,104.39,129.24,-19.227793
265,2024,2,82.38,139.93,70.39,106.7908,106.79,120.42,-11.318718
266,2024,3,92.89,144.37,88.47,114.6000,114.60,152.32,-24.763655
267,2024,4,100.30,143.80,107.53,118.4254,118.43,137.56,-13.906659
268,2024,5,103.72,182.93,102.00,137.0226,137.02,127.12,7.787917
269,2024,6,107.13,147.72,92.16,124.4772,124.48,117.25,6.166311
270,2024,7,102.95,182.17,107.38,136.1338,136.13,116.84,16.509757
271,2024,8,110.05,164.84,91.88,133.4252,133.43,132.57,0.648714
272,2024,9,102.27,145.04,107.38,120.1312,120.13,114.95,4.506307
273,2024,10,109.17,167.95,104.73,133.9464,133.95,129.53,3.412337
